<a href="https://colab.research.google.com/github/xmks-colab/xbox360/blob/main/GodXiso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title  baixar e extrair  { display-mode: "form" }
# Instalar dependências necessárias
!pip install pyunpack patool tqdm > /dev/null

import os
import zipfile
import shutil
import requests
from pyunpack import Archive
from tqdm.notebook import tqdm

from urllib.parse import parse_qs, urlparse, unquote

def baixar_arquivo():
    url = input("Cole a URL do arquivo para download: ").strip()
    nome_arquivo = url.split("/")[-1]

    # Corrigir o nome se tiver parâmetros
    query = urlparse(url).query
    params = parse_qs(query)
    if 'name' in params:
        nome_arquivo = unquote(params['name'][0])

    caminho_destino = os.path.join("/content", nome_arquivo)

    try:
        resposta = requests.get(url, stream=True)
        total = int(resposta.headers.get('content-length', 0))
        with open(caminho_destino, 'wb') as f, tqdm(
            total=total,
            unit='B',
            unit_scale=True,
            unit_divisor=1024
        ) as barra:
            for dados in resposta.iter_content(chunk_size=1024):
                f.write(dados)
                barra.update(len(dados))
        return caminho_destino
    except Exception as e:
        print("Erro ao baixar o arquivo:", e)
        return None

# ==============================
# Função: perguntar se deseja extrair
def perguntar_extrair():
    resposta = input("Deseja extrair o arquivo agora? (s/n): ").strip().lower()
    return resposta == 's'

# ==============================
# Função: extrair arquivos zip com barra de progresso
def extrair_zip(caminho, destino):
    with zipfile.ZipFile(caminho, 'r') as zip_ref:
        arquivos = zip_ref.namelist()
        for nome in tqdm(arquivos, desc="Extraindo ZIP", unit="arquivo"):
            zip_ref.extract(nome, destino)

# ==============================
# Função: extrair arquivos gerais (rar, etc.) com barra de progresso
def extrair_geral(caminho, destino):
    temp_dir = "/content/temp_extracao"
    os.makedirs(temp_dir, exist_ok=True)
    Archive(caminho).extractall(temp_dir)

    arquivos = []
    for raiz, _, nomes in os.walk(temp_dir):
        for nome in nomes:
            arquivos.append(os.path.join(raiz, nome))

    for caminho_abs in tqdm(arquivos, desc="Extraindo Arquivos", unit="arquivo"):
        caminho_rel = os.path.relpath(caminho_abs, temp_dir)
        destino_final = os.path.join(destino, caminho_rel)
        os.makedirs(os.path.dirname(destino_final), exist_ok=True)
        shutil.move(caminho_abs, destino_final)

    shutil.rmtree(temp_dir)

# ==============================
# Função principal
def main():
    caminho = baixar_arquivo()
    if caminho is None or not os.path.exists(caminho):
        return

    destino = "/content/extraido"
    os.makedirs(destino, exist_ok=True)

    if perguntar_extrair():
        ext = os.path.splitext(caminho)[1].lower()
        try:
            if ext == ".zip":
                extrair_zip(caminho, destino)
            else:
                extrair_geral(caminho, destino)
        except Exception as e:
            pass

# ==============================
# Executar
main()

In [ ]:

#@title Ambiente para extract-xiso { display-mode: "form" }
%%bash

# 1. Configurar arquitetura multiarch
sudo dpkg --add-architecture i386
sudo apt-get update -qq > /dev/null

# 2. Instalar dependências
sudo apt-get install -y \
    build-essential \
    gcc-multilib \
    g++-multilib \
    libc6:i386 \
    libstdc++6:i386 \
    zlib1g:i386 \
    lib32z1 \
    lib32stdc++6 \
    libx32gcc-s1 \
    libx32stdc++6 \
    libc6-dev-i386 > /dev/null 2>&1

# 3. Clonar e compilar
if [ ! -d "extract-xiso" ]; then
    git clone -q https://github.com/XboxDev/extract-xiso
fi

mkdir -p extract-xiso/build > /dev/null
cd extract-xiso/build

# 4. Compilar com flags 32-bit
compile_log=$(mktemp)
CFLAGS="-m32" CXXFLAGS="-m32" LDFLAGS="-m32" cmake .. > "$compile_log" 2>&1 && \
make >> "$compile_log" 2>&1 && \
chmod +x extract-xiso

# 5. Verificação robusta
if [ -f "extract-xiso" ]; then
    echo -e "\n✅ Compilação bem-sucedida!"
    file extract-xiso
    echo -e "\nComo usar:"
    echo "  ./extract-xiso/build/extract-xiso -x arquivo.iso"
else
    echo -e "\n❌ Falha na compilação. Verifique:"
    echo "1. Se todas dependências foram instaladas"
    echo "2. Detalhes da compilação:"
    cat "$compile_log" | grep -i 'error'
    exit 1
fi

# Remove log temporário
rm -f "$compile_log"

In [ ]:
#@title  Ambiente para iso2god-rs { display-mode: "form" }
# Instalar Rust silenciosamente
print("Instalando Rust...")
!curl https://sh.rustup.rs -sSf | sh -s -- -y > /dev/null 2>&1
print("Rust instalado com sucesso.")

# Atualizar o PATH manualmente
import os
os.environ["PATH"] += ":/root/.cargo/bin"

# Clonar o repositório iso2god-rs
print("Clonando repositório iso2god-rs...")
!git clone https://github.com/iliazeus/iso2god-rs.git > /dev/null 2>&1
print("Repositório clonado.")

# Entrar no diretório do projeto
%cd iso2god-rs

# Compilar o projeto em modo release
print("Compilando iso2god-rs...")
!cargo build --release > /dev/null 2>&1
print("Compilação concluída.")

# Mostrar onde está o executável
print("\nO executável está em: iso2god-rs/target/release/iso2god")

In [ ]:
#@title extract-xiso { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
import shutil

# Módulo 1: Lógica de construção de comandos
def construir_comando(modo, entrada, saida, ignorar_systemupdate):
    """Constroi o comando apropriado baseado nos parâmetros"""
    comando = ['./extract-xiso/build/extract-xiso']

    opcoes = {
        'Extrair ISO (-x)': {
            'flag': '-x',
            'suporte_s': True,
            'param_saida': ['-d', saida] if saida else []
        },
        'Criar ISO (-c)': {
            'flag': '-c',
            'suporte_s': False,
            'param_saida': [saida] if saida else []
        },
        'Reescrever ISO (-r)': {
            'flag': '-r',
            'suporte_s': False,
            'param_saida': []
        }
    }

    config = opcoes.get(modo)
    if not config:
        raise ValueError("Modo inválido selecionado")

    if ignorar_systemupdate and config['suporte_s']:
        comando.append('-s')

    comando.append(config['flag'])
    comando.append(entrada)
    comando.extend(config['param_saida'])

    return comando

# Módulo 2: Manipulação de arquivos
def mover_arquivo_reescrito(entrada, saida):
    """Move o arquivo reescrito para o destino apropriado"""
    nome_original = os.path.basename(entrada)

    if saida.lower().endswith('.iso'):
        diretorio_destino = os.path.dirname(saida)
        if diretorio_destino and not os.path.exists(diretorio_destino):
            os.makedirs(diretorio_destino)
        shutil.move(nome_original, saida)
        return saida
    else:
        if not os.path.exists(saida):
            os.makedirs(saida)
        destino = os.path.join(saida, nome_original)
        shutil.move(nome_original, destino)
        return destino

# Módulo 3: Execução principal
def executar_processamento(modo, entrada, saida, ignorar_systemupdate):
    """Orquestra todo o processo de conversão"""
    try:
        comando = construir_comando(modo, entrada, saida, ignorar_systemupdate)
        print('Executando:', ' '.join(comando))

        resultado = subprocess.run(comando, capture_output=True, text=True)

        if resultado.returncode == 0 and modo == 'Reescrever ISO (-r)' and saida:
            destino = mover_arquivo_reescrito(entrada, saida)
            print(f"ISO reescrita movida para: {destino}")

        return {
            'sucesso': resultado.returncode == 0,
            'saida': resultado.stdout,
            'erros': resultado.stderr
        }
    except Exception as e:
        return {
            'sucesso': False,
            'erros': str(e)
        }

# Módulo 4: Interface do usuário
def criar_interface():
    """Cria e gerencia a interface gráfica"""
    # Widgets
    modo_dropdown = widgets.Dropdown(
        options=['Extrair ISO (-x)', 'Criar ISO (-c)', 'Reescrever ISO (-r)'],
        value='Extrair ISO (-x)',
        description='Modo:'
    )

    entrada_input = widgets.Text(
        placeholder='/caminho/para/arquivo.iso',
        description='Entrada:'
    )

    saida_input = widgets.Text(
        placeholder='/caminho/de/saida',
        description='Saída:'
    )

    ignorar_checkbox = widgets.Checkbox(
        description='Ignorar $SystemUpdate'
    )

    output = widgets.Output()
    executar_btn = widgets.Button(description='Executar')

    # Lógica da interface
    def atualizar_interface(change):
        modo = change['new']
        saida_input.description = {
            'Extrair ISO (-x)': 'saída:',
            'Criar ISO (-c)': 'Caminho da ISO',
            'Reescrever ISO (-r)': 'Destino após reescrita'
        }.get(modo, 'Saída')

        ignorar_checkbox.disabled = modo != 'Extrair ISO (-x)'

    def ao_clicar(b):
        with output:
            output.clear_output()
            resultado = executar_processamento(
                modo=modo_dropdown.value,
                entrada=entrada_input.value.strip(),
                saida=saida_input.value.strip(),
                ignorar_systemupdate=ignorar_checkbox.value
            )

            if resultado['sucesso']:
                print("Processamento concluído com sucesso!")
                if resultado['saida']:
                    print(resultado['saida'])
            else:
                print("Ocorreu um erro:")
                print(resultado.get('erros', 'Erro desconhecido'))

    # Configurações
    modo_dropdown.observe(atualizar_interface, names='value')
    executar_btn.on_click(ao_clicar)
    atualizar_interface({'new': modo_dropdown.value})

    # Layout
    box_layout = widgets.Layout(
        width='80%',
        margin='10px 0'
    )

    for widget in [entrada_input, saida_input]:
        widget.layout = box_layout

    display(widgets.VBox([
        modo_dropdown,
        entrada_input,
        saida_input,
        ignorar_checkbox,
        executar_btn,
        output
    ]))

# Ponto de entrada principal
if __name__ == '__main__':
    criar_interface()

In [ ]:
#@title iso2god-rs { display-mode: "form" }
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess

def converter_iso2god(source_iso, dest_dir, usar_trim=False, game_title=None, num_threads=None, dry_run=False):
    iso2god_path = '/content/iso2god-rs/target/release/iso2god'

    if not os.path.isfile(source_iso):
        print(f"Erro: Arquivo ISO '{source_iso}' não encontrado.")
        return

    if not os.path.isfile(iso2god_path):
        print("Erro: iso2god não encontrado. Certifique-se de que foi compilado corretamente.")
        return

    if not os.path.exists(dest_dir) and not dry_run:
        os.makedirs(dest_dir)
        print(f"Pasta de destino '{dest_dir}' criada.")

    comando = [iso2god_path]

    if dry_run:
        comando.append("--dry-run")

    if usar_trim:
        comando.append("--trim")

    if game_title:
        comando.extend(["--game-title", game_title])

    if num_threads and num_threads > 0:
        comando.extend(["-j", str(num_threads)])

    comando.extend([source_iso, dest_dir])

    print(f"Executando comando: {' '.join(comando)}\n")

    try:
        resultado = subprocess.run(comando, capture_output=True, text=True, check=True)
        print(resultado.stdout)
    except subprocess.CalledProcessError as e:
        print("Erro ao executar iso2god:")
        print(e.stdout)
        print(e.stderr)

# Função que cria a interface GUI
def criar_interface_iso2god():
    """
    Cria a interface gráfica para selecionar ISO, pasta destino e opções no Colab.
    """
    # Widgets
    iso_input = widgets.Text(
        description='ISO:',
        placeholder='/content/seujogo.iso',
        layout=widgets.Layout(width='70%')
    )

    dest_input = widgets.Text(
        description='Destino:',
        placeholder='/content/saida_god',
        layout=widgets.Layout(width='70%')
    )

    trim_checkbox = widgets.Checkbox(
        description='Remover espaços (--trim)',
        value=False
    )

    game_title_input = widgets.Text(
        description='Título:',
        placeholder='Nome do Jogo (opcional)',
        layout=widgets.Layout(width='70%')
    )

    threads_slider = widgets.IntSlider(
        description='Threads:',
        min=1,
        max=16,
        value=4,
        step=1,
        layout=widgets.Layout(width='70%')
    )

    dry_run_checkbox = widgets.Checkbox(
        description='informações da iso(--dry-run)',
        value=False
    )

    botao_converter = widgets.Button(
        description='Executar',
        button_style='success'
    )

    output = widgets.Output()

    # Função interna para ação do botão
    def ao_clicar_converter(b):
        with output:
            clear_output()
            source_iso = iso_input.value.strip()
            dest_dir = dest_input.value.strip()
            usar_trim = trim_checkbox.value
            game_title = game_title_input.value.strip()
            if not game_title:
                game_title = None

            num_threads = threads_slider.value
            dry_run = dry_run_checkbox.value

            if source_iso and dest_dir:
                converter_iso2god(source_iso, dest_dir, usar_trim, game_title, num_threads, dry_run)
            else:
                print("Por favor, preencha todos os campos.")

    # Ligar o botão à função
    botao_converter.on_click(ao_clicar_converter)

    # Exibir tudo
    display(widgets.VBox([
        iso_input,
        dest_input,
        trim_checkbox,
        game_title_input,
        threads_slider,
        dry_run_checkbox,
        botao_converter,
        output
    ]))

# Chamando a interface para exibir no Colab
criar_interface_iso2god()